In [1]:
from sentence_transformers import SentenceTransformer , util
import torch
import numpy as np
import pandas as pd
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2') # sentence transormer
import spacy
# Download necessary nltk data
nlp = spacy.load('en_core_web_sm') # Spacy model that we use to partition sentences 


c:\Users\nima_\Documents\CompSci\CMPT 353\project\myenv\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
c:\Users\nima_\Documents\CompSci\CMPT 353\project\myenv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\nima_\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either nee

In [2]:
# Check if GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [28]:
# Function to chunk sentences and generate embeddings
def process_text(text):
    doc = nlp(text)
    sentences = [sent.text for sent in doc.sents]
    # Move sentences to device
    chunk_embeddings = model.encode(sentences, convert_to_tensor=True, device=device)
    story_embedding = torch.mean(chunk_embeddings, dim=0)
    return story_embedding.cpu()  # Move tensor back to CPU before returning

# Load the DataFrame from a JSON file
submissions = pd.read_json('output/filtered_and_balanced_submissions.json')
# Separate the DataFrame into the two categories
asshole_df = submissions[submissions['link_flair_text'] == 'Asshole']
not_asshole_df = submissions[submissions['link_flair_text'] == 'Not the A-hole']

# Determine the smaller of the two categories for balanced sampling
sample_size = min(len(asshole_df), len(not_asshole_df), 1000)  # do not exceed 1000 samples per class

# Sample from each category
asshole_sample = asshole_df.sample(sample_size).reset_index(drop=True)
not_asshole_sample = not_asshole_df.sample(sample_size).reset_index(drop=True)

# Combine the samples into a new DataFrame
balanced_sample = pd.concat([asshole_sample, not_asshole_sample]).reset_index(drop=True)
# Apply the function to each row, storing the result in a new column
balanced_sample['embedding'] = balanced_sample['selftext'].apply(process_text)

# sampled_submissions = submissions.sample(2000).reset_index(drop=True)
# sampled_submissions['embedding'] = balanced_sample['selftext'].apply(process_text)

# submissions['embedding'] = submissions['selftext'].sample(5).reset_index(drop=True).apply(process_text)
# Save the DataFrame as a pickle file
# submissions.to_pickle('submissions_with_embeddings.pkl')
balanced_sample.to_pickle('output/submissions_with_embeddings.pkl')
# print("DataFrame saved to submissions_with_embeddings.pkl")

In [29]:
balanced_sample['embedding'] 
# sampled_submissions.to_pickle('submissions_with_embeddings.pkl')


0       [tensor(0.0115), tensor(0.0421), tensor(-0.007...
1       [tensor(-0.0015), tensor(0.0372), tensor(-0.01...
2       [tensor(-0.0172), tensor(0.0516), tensor(-0.00...
3       [tensor(-0.0042), tensor(0.0340), tensor(-0.03...
4       [tensor(0.0022), tensor(0.0310), tensor(-0.013...
                              ...                        
1995    [tensor(0.0055), tensor(4.9448e-05), tensor(-0...
1996    [tensor(0.0150), tensor(0.0203), tensor(-0.019...
1997    [tensor(0.0022), tensor(0.0120), tensor(0.0018...
1998    [tensor(-0.0118), tensor(0.0315), tensor(-0.00...
1999    [tensor(-0.0123), tensor(0.0457), tensor(-0.00...
Name: embedding, Length: 2000, dtype: object